In [ ]:
# Install openCV if neccesary
#! pip install opencv-contrib-python

In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageFilter
from keras import models
import os
import tensorflow as tf

import matplotlib as mpl
import matplotlib.cm as mtpltcm
import matplotlib.pyplot as plt

In [ ]:
# Attempted realtime augmentation
def augment(input_image):
    image = Image.fromarray(input_image)
    imgGray = image.convert('L')
    imgGrayBlur = imgGray.filter(ImageFilter.BoxBlur(6))
    colormap = mpl.cm.jet
    cNorm = mpl.colors.Normalize(vmin=0, vmax=255)
    scalarMap = mtpltcm.ScalarMappable(norm=cNorm, cmap=colormap)
    colors = scalarMap.to_rgba(imgGrayBlur)
    image = np.array(colors)
    return image

In [ ]:
# Import Keras model from directory
model = models.load_model('Keras CNN Model')
video = cv2.VideoCapture(0)

# Activate OpenCV video and pass each frame through the model
while True:
        _, frame = video.read()

        #Convert frame to RGB
        
        im = Image.fromarray(frame) #,'RGB'

        #Resizing into dimensions
        im = im.resize((128,128))
        
        img_array = np.array(im)
        

        #Expand dimensions to match the 4D Tensor shape.
        img_array = np.expand_dims(img_array, axis=0)
        #img_array = augment(img_array)
        
        #Calling the predict using keras
        prediction = model.predict(img_array)#[0][0]
        
        text = 'Bad Prediction'
        #Create text overlay based on prediction
        #Take note from model wich label the prediction belongs to.
        
        # Get index of prediction array with highest value.
        max_index = np.argmax(prediction)
        
        if max_index == 0 and prediction[0][0] > 0.5:
            text = "Correct Mask"
            
        elif max_index == 1 and prediction[0][1] > 0.5:
            text = "Incorrect Mask"
            
            
        elif max_index == 2 and prediction[0][2] > 0.5:
            text = "No Mask"
                    

        # font
        font = cv2.FONT_HERSHEY_SIMPLEX
  
        # org
        org = (00, 185)
  
        # fontScale
        fontScale = 1
   
        # Change color based on predictions
        if text == "No Mask":
            color = (0, 0, 255)
        elif text == "Incorrect Mask":
            color = (255, 0, 0 )
        elif text == "Correct Mask":
            color = (0, 255, 0)
        else:
            color = ((128,128,128))
  
        # Line thickness of 2 px
        thickness = 2
   
        # Using cv2.putText() method
        frame2 = cv2.putText(frame, text, org, font, fontScale, 
                 color, thickness, cv2.LINE_AA, False)   
        cv2.imshow('Prediction', frame2)
        key=cv2.waitKey(1)
        if key == ord('q'): # Press q to quit video feed loop.
                break
video.release()
cv2.destroyAllWindows()